# Scalping Strategy Test

> Strategy 1

Daily Candles (EUR/UDS)

**Links**:

Library TA: https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html

API Data: https://www.alphavantage.co/documentation/


In [1]:
import requests
from pprint import pprint
import pandas as pd
from datetime import date
import json

# Get data from Alpha Vantage 
#url = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey=9H93UG738ET21IS4'
symbol = 'EURUSD'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={symbol[:3]}&to_symbol={symbol[3:]}&outputsize=full&apikey=9H93UG738ET21IS4"
r = requests.get(url)
data = r.json()['Time Series FX (Daily)']
df = pd.DataFrame.from_dict(data,orient='index').astype('float')

# Rename columns
df.rename(columns = {'1. open':'open', '2. high':'high', '3. low':'low', '4. close':'close',}, inplace = True)

# Sort values by time (ascending)
df.sort_index(inplace=True)

# Select Dates
df = df.loc['2021-01-01':'2022-12-31']

print(df.head(3))

               open     high      low    close
2021-01-01  1.22150  1.22150  1.22138  1.22138
2021-01-04  1.22386  1.23098  1.22290  1.22461
2021-01-05  1.22469  1.23057  1.22428  1.22957


In [4]:
import ta

# Candle color
df['candleColor'] = df.apply(lambda row: 'GREEN' if row.close > row.open else 'RED' , axis=1)

# Candle body
df['candleBody'] = abs(df['open']-df['close'])

# Parabolic SAR + indicator of UP/DOWN
df['parabolicSar'] = ta.trend.PSARIndicator(high= df['high'], low= df['low'], close=df['close'], step= 0.02, max_step= 0.2, fillna= False).psar()
df['parabolicIndicator'] = df.apply(lambda row: 'ABOVE' if row.parabolicSar > row.close else 'BELOW' , axis=1)

# Bollinger Bands
df['bolHigh'] = ta.volatility.bollinger_hband(df['close'], window=20, window_dev=2, fillna=False)
df['bolLow'] = ta.volatility.bollinger_lband(df['close'], window=20, window_dev=2, fillna=False)

# Skip first 20 rows used to calculate Bollinger bands
df = df.iloc[19:,:]

df = df.reset_index().rename(columns = {'index':'time'})

print(df)

    time        time     open     high     low    close candleColor  \
0     19  2021-02-24  1.21496  1.21747  1.2107  1.21664       GREEN   
1     20  2021-02-25  1.21641  1.22432  1.2154  1.21740       GREEN   
2     21  2021-02-26  1.21718  1.21841  1.2060  1.20704         RED   
3     22  2021-03-01  1.20718  1.21012  1.2026  1.20492         RED   
4     23  2021-03-02  1.20484  1.20945  1.1990  1.20900       GREEN   
..   ...         ...      ...      ...     ...      ...         ...   
327  346  2022-05-27  1.07292  1.07652  1.0695  1.07317       GREEN   
328  347  2022-05-30  1.07326  1.07870  1.0724  1.07770       GREEN   
329  348  2022-05-31  1.07762  1.07794  1.0677  1.07313         RED   
330  349  2022-06-01  1.07308  1.07393  1.0625  1.06519         RED   
331  350  2022-06-02  1.06531  1.07339  1.0643  1.07308       GREEN   

     candleBody  parabolicSar parabolicIndicator   bolHigh    bolLow  
0       0.00168      1.205744              BELOW  1.219561  1.199146  
1    

> Strategy

Missing points...

position = NEUTRAL
- 1. Buy if BELOW + GREEN and Current body < 2x Previous Body -> Long, Set Stop Loss (Current Low - 0.0001 x50)
- 2. Sell if ABOVE + RED and Current body < 2x Previous Body -> Short, Set Stop Loss (Current High + 0.0001 x50)
- 3. Else: Mantain

position = LONG
- 1. Sell if ABOVE + RED and Current body < 2x Previous Body -> NEUTRAL+SHORT
- 2. Sell if ABOVE -> NEUTRAL
- 3. Check Stop Loss -> if Stop Loss > Current Close -> Close Position -> NEUTRAL

- 4. Update Stop Loss if New Close > Previous Close -> Set Stop Loss (Current Low - 0.0001 x50)
- 5. Else: Mantain

position = SHORT
- 1. Buy if BELOW + GREEN and Current body < 2x Previous Body -> NEUTRAL+LONG
- 2. Buy if BELOW -> NEUTRAL
- 3. Check Stop Loss -> if Stop Loss < Current Close -> Close Position -> NEUTRAL

- 4. Update Stop Loss if New Close < Previous Close -> Set Stop Loss (Current High + 0.0001 x50)
- 5. Else: Mantain


In [3]:
# Algorithm
pip = 0.0001

# Future data
future = df.iloc[1:,:]

# Position
position = 'Neutral'
price = 0
stopLoss = 0

sarIndicator = None

# Stop Loss / Take Profit
slPips = pip*150
tpRatio = pip*150

# Balance
p1 = 0
balance = 1000
perc = 1

# DATA FRAME DATA
d = {'time':[],'position':[],'price':[],'stopLoss':[],'balance':[]}

def updateDic(time,position,closePrice,stopLoss,index,balance):
    # DataFrame data dictionary
    d['time'].append(time)
    d['position'].append(position)
    d['price'].append(closePrice)
    d['stopLoss'].append(stopLoss)
    d['balance'].append(balance)
    # Print
    #print(f"Time: {row['time']} -> {position} @ Price = {price}, TL = {stopLoss}")

for index, row in future.iterrows():
    # Previous Update
    previous = df.iloc[index-1:index,:].to_dict('records')[0]
    
    if position == 'Neutral':
        # See if there has been a change in Trend after SL or TP
        if sarIndicator == None or sarIndicator != row['parabolicIndicator']:
            sarIndicator = None
            # Go Long
            if row['parabolicIndicator'] == 'BELOW' and row['candleColor'] == 'GREEN' and row['candleBody'] < 2*previous['candleBody']:
                position = 'Long'
                price = row['close']
                stopLoss = row['low'] - slPips

                p1 = row['close']
                updateDic(row['time'],position,row['close'],stopLoss,index,balance)
                continue
            
            # Go Long 2
            elif row['parabolicIndicator'] == 'BELOW' and row['candleColor'] == 'RED' and previous['candleColor'] == 'GREEN' and previous['candleBody'] > 1.5*row['candleBody']:
                position = 'Long'
                price = row['close']
                stopLoss = row['low'] - slPips

                p1 = row['close']
                updateDic(row['time'],position,row['close'],stopLoss,index,balance)
                continue

            # Go Short
            elif row['parabolicIndicator'] == 'ABOVE' and row['candleColor'] == 'RED' and row['candleBody'] < 2*previous['candleBody']:
                position = 'Short'
                price = row['close']
                stopLoss = row['high'] + slPips

                p1 = row['close']
                updateDic(row['time'],position,row['close'],stopLoss,index,balance)
                continue
            
            # Go Short 2
            elif row['parabolicIndicator'] == 'ABOVE' and row['candleColor'] == 'GREEN' and previous['candleColor'] == 'RED' and previous['candleBody'] > 1.5*row['candleBody']:
                position = 'Short'
                price = row['close']
                stopLoss = row['high'] + slPips

                p1 = row['close']
                updateDic(row['time'],position,row['close'],stopLoss,index,balance)
                continue

            else: # Mantain
                price = row['close']
                updateDic(row['time'],position,row['close'],stopLoss, index,balance)
                continue
                
        # No change in trend after SL or TP
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],stopLoss, index,balance)
            continue
            
    elif position == 'Long':
        
        # If Short trigger - Neutral then Short
        if row['parabolicIndicator'] == 'ABOVE' and row['candleColor'] == 'RED' and row['candleBody'] < 2*previous['candleBody']:
            position = 'Short'
            price = row['close']
            stopLoss = row['high'] + slPips
            
            balance = perc*balance*(row['close']-p1) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # If Short trigger - Neutral then Short 2
        elif row['parabolicIndicator'] == 'ABOVE' and row['candleColor'] == 'GREEN' and previous['candleColor'] == 'RED' and previous['candleBody'] > 1.5*row['candleBody']:
            position = 'Short'
            price = row['close']
            stopLoss = row['high'] + slPips
            
            balance = perc*balance*(row['close']-p1) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # If Above - Close position (change in trend)
        elif row['parabolicIndicator'] == 'ABOVE':
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = perc*balance*(row['close']-p1) + balance
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
            
        # Check TP
        elif row['close'] > p1 + tpRatio:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            # Indicator to open next position if change in TREND
            #sarIndicator = row['parabolicIndicator']
            
            balance = perc*balance*(row['close']-p1) + balance
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
            
        # Check Stop Loss
        elif stopLoss > row['close']:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            # Indicator to open next position if change in TREND
            #sarIndicator = row['parabolicIndicator']
            
            balance = perc*balance*(row['close']-p1) + balance
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # Update Stop Loss
        elif row['close'] > previous['close']: 
            stopLoss = row['low'] - slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        else: # Mantain
            price = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
    
    elif position == 'Short': 
        # If Long trigger - Neutral then Long
        if row['parabolicIndicator'] == 'BELOW' and row['candleColor'] == 'GREEN' and row['candleBody'] < 2*previous['candleBody']:
            position = 'Long'
            price = row['close']
            stopLoss = row['low'] - slPips
            
            balance = perc*balance*(p1-row['close']) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
            
        # If Long trigger - Neutral then Long 2
        elif row['parabolicIndicator'] == 'BELOW' and row['candleColor'] == 'RED' and previous['candleColor'] == 'GREEN' and previous['candleBody'] > 1.5*row['candleBody']:
            position = 'Long'
            price = row['close']
            stopLoss = row['low'] - slPips
            
            balance = perc*balance*(p1-row['close']) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # If Below - Close position (change in trend)
        elif row['parabolicIndicator'] == 'BELOW':
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = perc*balance*(p1-row['close']) + balance
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # Check TP
        elif row['close'] < p1 - tpRatio:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            # Indicator to open next position if change in TREND
            #sarIndicator = row['parabolicIndicator']
            
            balance = perc*balance*(p1-row['close']) + balance
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
            
        # Check Stop Loss
        elif stopLoss < row['close']:
            position = 'Neutral'
            price = row['close']
            
            # Indicator to open next position if change in TREND
            #sarIndicator = row['parabolicIndicator']
            
            balance = perc*balance*(p1-row['close']) + balance
            stopLoss = 0
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        # Update Stop Loss
        elif row['close'] < previous['close']: 
            stopLoss = row['high'] + slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
        
        else: # Mantain
            price = row['close']
            updateDic(row['time'],position,row['close'],stopLoss,index,balance)
            continue
    
    
print(balance)
#dfTest = pd.DataFrame.from_dict(d)
#dfTest.to_excel('testGoodProfit.xlsx')
    

1151.0957502775016
